# Module 5: Assignment - Comparison Exercise - Analyzing Deep Learning Frameworks

In [4]:
# Import Tensor Flow
import tensorflow
# Import Keras
from tensorflow import keras
# Import Numpy
import numpy as np
# Import Matplotlib
import matplotlib.pyplot as plt

# Load the bird identification dataset from Kaggle https://www.kaggle.com/datasets/gpiosenka/100-bird-species
# Load the dataset
train_data = keras.utils.image_dataset_from_directory('birds/train', image_size=(224, 224), batch_size=32)
test_data = keras.utils.image_dataset_from_directory('birds/test', image_size=(224, 224), batch_size=32)
validation_data = keras.utils.image_dataset_from_directory('birds/valid', image_size=(224, 224), batch_size=32)
# Load the class names from csv file
class_names = np.loadtxt('birds/birds.csv', delimiter=',', usecols=(1,), dtype=str, skiprows=1)
# Print the class names
print(class_names)
# Print the shape of the training data
print(train_data)
# Print the shape of the test data
print(test_data)
# Print the shape of the validation data
print(validation_data)

# Create a CNN model
model = keras.Sequential([
    keras.layers.Conv2D(32, 3, activation='relu', input_shape=(224, 224, 3)),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(32, 3, activation='relu'),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(32, 3, activation='relu'),
    keras.layers.MaxPooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(250, activation='softmax')
])
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Fit the model
model.fit(train_data, epochs=10, validation_data=validation_data)

# Evaluate the model
model.evaluate(test_data)

# Save the model
model.save('birds_model')

# Load the model
loaded_model = keras.models.load_model('birds_model')

# Make predictions
predictions = loaded_model.predict(test_data)
# Print the predictions
print(predictions)


Found 84635 files belonging to 525 classes.
Found 2625 files belonging to 525 classes.
Found 2625 files belonging to 525 classes.
['train/ABBOTTS BABBLER/001.jpg' 'train/ABBOTTS BABBLER/007.jpg'
 'train/ABBOTTS BABBLER/008.jpg' ... 'valid/BLACK BREASTED PUFFBIRD/1.jpg'
 'valid/BLACK BREASTED PUFFBIRD/2.jpg'
 'valid/BLACK BREASTED PUFFBIRD/5.jpg']
<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>
<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>
<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>
Epoch 1/10


2024-02-21 03:18:24.096956: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8907
2024-02-21 03:18:24.200725: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-02-21 03:18:24.647234: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-02-21 03:18:24.685078: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x9ddfcd0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-21 03:18:24.685092: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A1000 Laptop GPU, Compute Capability 8.6
2024-02-21 03:18:24.688904: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-02-21 03:18:24.731655

   1/2645 [..............................] - ETA: 2:06:50 - loss: nan - accuracy: 0.0000e+00

2024-02-21 03:18:26.337924: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 772.89MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


2645/2645 [==============================] - 99s 36ms/step - loss: nan - accuracy: 0.0019 - val_loss: nan - val_accuracy: 0.0019
Epoch 2/10
2645/2645 [==============================] - 95s 36ms/step - loss: nan - accuracy: 0.0019 - val_loss: nan - val_accuracy: 0.0019
Epoch 3/10
2645/2645 [==============================] - 95s 36ms/step - loss: nan - accuracy: 0.0019 - val_loss: nan - val_accuracy: 0.0019
Epoch 4/10
 405/2645 [===>..........................] - ETA: 1:18 - loss: nan - accuracy: 0.0019